In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from skimage.filters import threshold_sauvola

In [4]:
def apply_convex_hull(mask):
    # Temukan kontur
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    hull_mask = np.zeros_like(mask)

    for cnt in contours:
        hull = cv2.convexHull(cnt)
        cv2.drawContours(hull_mask, [hull], -1, 255, -1)  # isi hull
    return hull_mask

def detect_lungs_grid2(image_path, outdir=r"D:\project\python\2025\tbcdetect\_streamlit\data\TB_Train\Normal"):
    
    file_name = os.path.basename(image_path)
    steps = []   # SIMPAN SEMUA STEP

    # -----------------------------
    # 1. Load + Resize + Border
    # -----------------------------
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Gagal membaca gambar, periksa path input.")

    canvas_height = 512
    canvas_width = 512
    img = cv2.resize(img, (canvas_width, canvas_height))

    pad = 5
    img = cv2.copyMakeBorder(img, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=(0,0,0))
    steps.append(("01. Original + Border", img.copy(), None))

    # -----------------------------
    # 2. Canny Bottom
    # -----------------------------
    grayxx = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    otsu_val, _ = cv2.threshold(grayxx, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    lower = int(otsu_val)
    upper = 200

    edges_full = cv2.Canny(img, lower, upper)
    h = edges_full.shape[0]

    cut_start = int(0.50 * h)
    edges_bottom = np.zeros_like(edges_full)
    edges_bottom[cut_start:, :] = edges_full[cut_start:, :]
    steps.append(("02. Canny Bottom", edges_bottom.copy(), "gray"))

    edge_cl = np.zeros_like(img)
    edge_cl[edges_bottom != 0] = (0,255,255)
    overlay_edges = cv2.addWeighted(img, 0.97, edge_cl, 0.15, 0)

    gray0 = cv2.cvtColor(overlay_edges, cv2.COLOR_BGR2GRAY)
    steps.append(("03. Overlay Edges", overlay_edges.copy(), None))
    #steps.append(("04. Gray After Overlay", gray0.copy(), "gray"))

    # -----------------------------
    # 3. CLAHE
    # -----------------------------
    clahe = cv2.createCLAHE(2.0, (3,3))
    gray_clahe = clahe.apply(gray0)
    steps.append(("04. CLAHE", gray_clahe.copy(), "gray"))

    # -----------------------------
    # 4. Canny after CLAHE
    # -----------------------------
    otsu2, _ = cv2.threshold(gray_clahe, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    lower2 = int(otsu2 * 0.5)
    upper2 = int(otsu2 * 1.5)
    
    edges_after = cv2.Canny(gray_clahe, lower2, upper2)
    steps.append(("05a. Canny After CLAHE", edges_after.copy(), "gray"))
    
    h, w = edges_after.shape
    
    # Persentase tepi
    top_pct = 0.21       # 20% atas
    bottom_pct = 0.25    # 25% bawah
    left_pct = 0.1      # 5% kiri
    right_pct = 0.1     # 5% kanan
    
    # Hitung indeks
    top_cut = int(top_pct * h)
    bottom_cut = int((1 - bottom_pct) * h)
    left_cut = int(left_pct * w)
    right_cut = int((1 - right_pct) * w)
    
    # Buat mask kosong
    edges_tb = np.zeros_like(edges_after)
    
    # Salin tepi atas dan bawah
    edges_tb[:top_cut, :] = edges_after[:top_cut, :]
    edges_tb[bottom_cut:, :] = edges_after[bottom_cut:, :]
    
    # Salin tepi kiri dan kanan
    edges_tb[:, :left_cut] = edges_after[:, :left_cut]
    edges_tb[:, right_cut:] = edges_after[:, right_cut:]
    
    # Dilation untuk memperkuat tepi
    edges_tb = cv2.dilate(edges_tb, cv2.getStructuringElement(cv2.MORPH_RECT, (3,3)))
    steps.append(("05b. Top20 + Bot25 + lr 5", edges_tb.copy(), "gray"))

    edge_cl2 = np.zeros_like(img)
    edge_cl2[edges_tb != 0] = (0,255,255)

    tmp = cv2.cvtColor(gray_clahe, cv2.COLOR_GRAY2BGR)
    overlay_tb = cv2.addWeighted(tmp, 0.97, edge_cl2, 0.50, 0)
    #steps.append(("05d. Overlay TopBottom Edges", overlay_tb.copy(), None))

    gray2 = cv2.cvtColor(overlay_tb, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.createCLAHE(2.0, (3,3)).apply(gray2)

    # -----------------------------
    # 5. Mean Shift
    # -----------------------------
    tmp2 = cv2.cvtColor(gray2, cv2.COLOR_GRAY2BGR)
    ms = cv2.pyrMeanShiftFiltering(tmp2, sp=15, sr=30)
    gray_blur = cv2.cvtColor(ms, cv2.COLOR_BGR2GRAY)
    steps.append(("06. MeanShift", gray_blur.copy(), "gray"))

    # -----------------------------
    # 6. Threshold
    # -----------------------------
    otsu_val2, _ = cv2.threshold(gray_blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    _, thresh = cv2.threshold(gray_blur, otsu_val2, 255, cv2.THRESH_BINARY_INV)
    steps.append(("07. Threshold", thresh.copy(), "gray"))

    # -----------------------------
    # 7. Morphology
    # -----------------------------
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    eroded = cv2.erode(thresh,kernel , iterations=2)
    
    kernel7 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
    morphed = cv2.morphologyEx(eroded, cv2.MORPH_OPEN, kernel7, iterations=2)
    steps.append(("08. Morphology", morphed.copy(), "gray"))

    # -----------------------------
    # 8. Connected Components
    # -----------------------------
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(morphed, connectivity=8)
    h_img, w_img = morphed.shape
    lung_candidates = []

    for i in range(1, num_labels):
        x, y, w_i, h_i, area = stats[i]

        # buang sudut
        batas = 5
        touches = (
            (x<=batas and y<=batas) or
            (x+w_i>=w_img-batas and y<=batas) or
            (x<=batas and y+h_i>=h_img-batas) or
            (x+w_i>=w_img-batas and y+h_i>=h_img-batas)
        )
        if touches:
            continue

        # buang border
        BORDER = 5
        if (y<=BORDER or y+h_i>=h_img-BORDER or x<=BORDER or x+w_i>=w_img-BORDER):
            continue

        aspect_ratio = h_i / (w_i + 1e-5)
        if area > 3000 and 0.5 < aspect_ratio < 6.0:
            lung_candidates.append((i, area))

    # -----------------------------
    # 9. KIRI / KANAN 3 TERBESAR + CENTER OF GRAVITY
    # -----------------------------
    W = w_img
    H = h_img
    center_x = W // 2
    center_y = H // 2

    candidates_left = []
    candidates_right = []

    for comp_id, area in lung_candidates:
        cx, cy = centroids[comp_id]
        if cx <= center_x:
            candidates_left.append((comp_id, area, cx, cy))
        elif cx >= center_x:
            candidates_right.append((comp_id, area, cx, cy))

    # Ambil 3 terbesar
    candidates_left = sorted(candidates_left, key=lambda x: x[1], reverse=True)[:3]
    candidates_right = sorted(candidates_right, key=lambda x: x[1], reverse=True)[:3]

    # Pilih yang centroid paling dekat ke tengah
    def pick_most_centered(candidates, side="left"):
        if not candidates:
            return None
        if side=="left":
            ref_x = W * 0.25
        else:
            ref_x = W * 0.75
        ref_y = H * 0.4
        dist = [(comp_id, np.sqrt((cx-ref_x)**2 + (cy-ref_y)**2)) for comp_id, _, cx, cy in candidates]
        dist_sorted = sorted(dist, key=lambda x: x[1])
        return dist_sorted[0][0]

    #seleksi kandidat
    candidate_overlay = img.copy()
    alpha = 0.7
    
    # Variasi warna kiri (merah)
    reds = [(224,17,95),    # Ruby
    (255,127,80),   # Coral
    (255,36,0)     # Scarlet
           ]      
    
    # Variasi warna kanan (biru)
    blues = [(95,17,224),    # Ruby
    (80,127,255),   # Coral
    (0,36,255),     # Scarlet
            ]      
    
    for idx, (comp_id, _, _, _) in enumerate(candidates_left):
        mask_tmp = np.zeros_like(morphed)
        mask_tmp[labels == comp_id] = 255
        mask_color = np.zeros_like(img)
        color = reds[idx % len(reds)]
        mask_color[mask_tmp != 0] = color
        candidate_overlay = cv2.addWeighted(candidate_overlay, 1, mask_color, alpha, 0)
    
    for idx, (comp_id, _, _, _) in enumerate(candidates_right):
        mask_tmp = np.zeros_like(morphed)
        mask_tmp[labels == comp_id] = 255
        mask_color = np.zeros_like(img)
        color = blues[idx % len(blues)]
        mask_color[mask_tmp != 0] = color
        candidate_overlay = cv2.addWeighted(candidate_overlay, 1, mask_color, alpha, 0)
    
    steps.append(("9a. Candidates Left/Right Overlay", candidate_overlay.copy(), None))
    
    left_id  = pick_most_centered(candidates_left, side="left")
    right_id = pick_most_centered(candidates_right, side="right")

    mask_left = np.zeros_like(morphed)
    mask_right = np.zeros_like(morphed)

    if left_id is not None:
        mask_left[labels == left_id] = 255
    if right_id is not None:
        mask_right[labels == right_id] = 255

    # -----------------------------
    # 10. Convex Hull & Masked
    # -----------------------------
    mask_left_hull = apply_convex_hull(mask_left)
    mask_right_hull = apply_convex_hull(mask_right)
    hull_mask = cv2.bitwise_or(mask_left_hull, mask_right_hull)
    masked = cv2.bitwise_and(img, img, mask=hull_mask)

    overlay_hull = img.copy()
    overlay_color = (0, 255, 0)
    alpha = 0.3
    overlay_hull[mask_left_hull != 0] = (1-alpha)*overlay_hull[mask_left_hull != 0] + alpha*np.array(overlay_color)
    overlay_hull[mask_right_hull != 0] = (1-alpha)*overlay_hull[mask_right_hull != 0] + alpha*np.array(overlay_color)
    steps.append(("09b. Convex Hull Overlay", overlay_hull.copy(), None))
    left_img  = cv2.bitwise_and(img, img, mask=mask_left_hull)
    right_img = cv2.bitwise_and(img, img, mask=mask_right_hull)
    steps.append(("10. Convex Hull Masked", masked.copy(), None))

    # -----------------------------
    # 11. Visualisasi semua step
    # -----------------------------
    

    # --- Tampilkan grid 3x3 (atau lebih) ---
    #
    #n = len(steps)
    #cols = 3
    #rows = int(np.ceil(n/cols))
    #plt.figure(figsize=(15, 5*rows))

    
    #for i,(title,img,cmap) in enumerate(steps):
    #    plt.subplot(rows, cols, i+1)
    #    if img.ndim==2:
    #        plt.imshow(img, cmap=cmap if cmap else "gray")
    #    else:
    #        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #    plt.title(title)
    #    plt.axis("off")
    
    #plt.figure(figsize=(15, 5 * len(steps)))
    

    #plt.tight_layout()
    #plt.show()

    # Simpan hasil akhir
    cv2.imwrite(os.path.join(outdir, file_name), masked)
    #print(f"Hasil akhir disimpan di: {os.path.abspath(outdir)}")

In [ ]:
template = create_lung_template(r"D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Normal\Normal-1817.png")

In [14]:
## normal
image_folder = r"D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Normal"
##image_folder = r"D:\project\python\2025\tbcdetect\_data_shenzhen\normal"
all_files = os.listdir(image_folder)

# Filter for .png only
png_files = [f for f in all_files if f.lower().endswith('.png')]

In [10]:
for f in png_files:
    patth=os.path.join(image_folder, f)
    print(patth)
    detect_lungs_grid2(patth,r"D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Train\normal")

D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0327_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0328_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0329_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0330_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0331_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0332_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0333_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0334_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0335_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0336_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0337_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0338_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\normal\CHNCXR_0339_1.png
D:\project\python\2025\tbcdetect\_data_shenzhen\nor

In [11]:
#image_folder = r"D:\project\python\2025\tbcdetect\_data_shenzhen\tb"
image_folder = r"D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Tuberculosis"
all_files = os.listdir(image_folder)

# Filter for .png only
png_files = [f for f in all_files if f.lower().endswith('.png')]

In [12]:
for f in png_files:
    patth=os.path.join(image_folder, f)
    detect_lungs_grid2(patth,r"D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Train\tb")

In [6]:
#paralel normal
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

image_folder = r"D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Normal"
png_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]

def process_file(f):
    patth = os.path.join(image_folder, f)
    # panggil fungsi deteksi / ekstraksi fitur
    detect_lungs_grid2(patth, r"D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Train\normal")
    return f  # optional: bisa return hasil

# Gunakan ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=8) as executor:  # sesuaikan dengan jumlah core CPU
    futures = [executor.submit(process_file, f) for f in png_files]
    for future in as_completed(futures):
        print(f"{future.result()} selesai")

Normal-1003.png selesai
Normal-1002.png selesai
Normal-1001.png selesai
Normal-1.png selesai
Normal-100.png selesai
Normal-1004.png selesai
Normal-1000.png selesai
Normal-10.png selesai
Normal-1005.png selesai
Normal-1009.png selesai
Normal-1010.png selesai
Normal-1006.png selesai
Normal-1008.png selesai
Normal-1007.png selesai
Normal-101.png selesai
Normal-1011.png selesai
Normal-1012.png selesai
Normal-1014.png selesai
Normal-1015.png selesai
Normal-1013.png selesai
Normal-1017.png selesai
Normal-1016.png selesai
Normal-1018.png selesai
Normal-1019.png selesai
Normal-102.png selesai
Normal-1020.png selesai
Normal-1021.png selesai
Normal-1022.png selesai
Normal-1024.png selesai
Normal-1023.png selesai
Normal-1025.png selesai
Normal-1026.png selesai
Normal-1028.png selesai
Normal-1027.png selesai
Normal-1029.png selesai
Normal-1030.png selesai
Normal-103.png selesai
Normal-1032.png selesai
Normal-1031.png selesai
Normal-1033.png selesai
Normal-1035.png selesai
Normal-1034.png selesai
N

In [5]:
#paralel tb
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

image_folder = r"D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Tuberculosis"
png_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]

def process_file(f):
    patth = os.path.join(image_folder, f)
    # panggil fungsi deteksi / ekstraksi fitur
    detect_lungs_grid2(patth, r"D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Train\tb")
    return f  # optional: bisa return hasil

# Gunakan ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=8) as executor:  # sesuaikan dengan jumlah core CPU
    futures = [executor.submit(process_file, f) for f in png_files]
    for future in as_completed(futures):
        print(f"{future.result()} selesai")

Tuberculosis-1.png selesai
Tuberculosis-10.png selesai
Tuberculosis-100.png selesai
Tuberculosis-104.png selesai
Tuberculosis-102.png selesai
Tuberculosis-101.png selesai
Tuberculosis-105.png selesai
Tuberculosis-103.png selesai
Tuberculosis-106.png selesai
Tuberculosis-107.png selesai
Tuberculosis-108.png selesai
Tuberculosis-109.png selesai
Tuberculosis-110.png selesai
Tuberculosis-111.png selesai
Tuberculosis-11.png selesai
Tuberculosis-112.png selesai
Tuberculosis-113.png selesai
Tuberculosis-114.png selesai
Tuberculosis-115.png selesai
Tuberculosis-119.png selesai
Tuberculosis-117.png selesai
Tuberculosis-116.png selesai
Tuberculosis-118.png selesai
Tuberculosis-12.png selesai
Tuberculosis-120.png selesai
Tuberculosis-121.png selesai
Tuberculosis-123.png selesai
Tuberculosis-122.png selesai
Tuberculosis-125.png selesai
Tuberculosis-124.png selesai
Tuberculosis-126.png selesai
Tuberculosis-127.png selesai
Tuberculosis-128.png selesai
Tuberculosis-129.png selesai
Tuberculosis-13.png